# Purity Computation and validation Plots
---

## Definition of Purity

The definition purity is: "The number of correct-matched top divided by the total number"(in events or entries.)<br/><br/>
We can define two types of putiry:<br/>

1. $P_{top}$: This is a purity base on the number of entries.<br/>

    $P_{top} = \frac{2*N_{both correct} + N_{one correct}}{N_{both correct} + N_{one correct} + N_{Neither correct}}$

2. $P_{event}$:  This is a purity base on the number of events.<br/>

    $P_{event} = \frac{N_{both correct}}{N_{both correct} + N_{one correct} + N_{Neither correct}}$

## Definition of "Correct"

Assume we have 2 candidates top pair obtained from chi2 reconstruction. We can compare this 2 candidates with the truth matching data(obtained from $\Delta$R matching). 
<br/>
For example, <br/> 
If we have a set of truth matching result.<br/>
Truth matching result: Candidate 1: (0, 1, 4), Candidate 2: (5, 3, 2)<br/>
<br/>
We called a candidates obtained from chi2 reconstruction "correct" if Chi2 candidate: (0, 1, 4)<br/>
We called a candidates obtained from chi2 reconstruction "incorrect" if Chi2 candidate: (0, 3, 2) (The quarks are tagged to the wrong candidate) <br/>
We called a candidates obtained from chi2 reconstruction "non-match" if Chi2 candidate: (0, 1, 7) (A wrong quark appears in the candidate) <br/>

<br/>
In the definition upon, we will have 9 possibilities. <br/>
For the convenience, we seperate them into 4 cases:

1. Case 1: Both correct.
2. Case 2: 1 correct + 1 incorrect.
3. Case 3: Neither correct.
4. Case 4: 1 or 2 NOn-matched candidate exist.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import h5py, tqdm
from script.utilize import delta_R, deltaPhi

PREFIX_chi2 = "output/ttbar_output/"
chi2_file = ""
seq_chi2 = (PREFIX_chi2, "event_record_", "chi2_", "normal", ".h5")

with h5py.File(chi2_file.join(seq_chi2), 'r') as f:

        N_match_top_in_event_chi2 = f['N_match_top_in_event'][:]
        jet_pt_chi2 = f['jet_pt'][:]
        jet_eta_chi2 = f['jet_eta'][:]
        jet_phi_chi2 = f['jet_phi'][:]
        jet_mass_chi2 = f['jet_mass'][:]
        jet_btag_chi2 = f['jet_btag'][:]
        jet_barcode_chi2 = f['jet_barcode'][:]
        parton_pt_chi2 = f['parton_pt'][:]
        parton_eta_chi2 = f['parton_eta'][:]
        parton_phi_chi2 = f['parton_phi'][:]
        parton_mass_chi2 = f['parton_mass'][:]
        jet_parton_index_chi2 = f['jet_parton_index'][:]
        parton_jet_index_chi2 = f['parton_jet_index'][:]
        chi2_value = f['Chi2_value'][:]

PREFIX_chi2_mtop = "output/ttbar_output/"
chi2_file = ""
seq_chi2_mtop = (PREFIX_chi2, "event_record_", "chi2_", "mtop", ".h5")

with h5py.File(chi2_file.join(seq_chi2_mtop), 'r') as f:
    N_match_top_in_event_chi2_mtop = f['N_match_top_in_event'][:]
    jet_pt_chi2_mtop = f['jet_pt'][:]
    jet_eta_chi2_mtop = f['jet_eta'][:]
    jet_phi_chi2_mtop = f['jet_phi'][:]
    jet_mass_chi2_mtop = f['jet_mass'][:]
    jet_btag_chi2_mtop = f['jet_btag'][:]
    jet_barcode_chi2_mtop = f['jet_barcode'][:]
    parton_pt_chi2_mtop = f['parton_pt'][:]
    parton_eta_chi2_mtop = f['parton_eta'][:]
    parton_phi_chi2_mtop = f['parton_phi'][:]
    parton_mass_chi2_mtop = f['parton_mass'][:]
    jet_parton_index_chi2_mtop = f['jet_parton_index'][:]
    parton_jet_index_chi2_mtop = f['parton_jet_index'][:]
    chi2_value_mtop = f['Chi2_value'][:]

N_match_top_in_event_del = []
jet_parton_index_del = []
parton_jet_index_del = []
jet_pt_del = []
jet_phi_del = []
jet_eta_del = []
jet_barcode_del = []
jet_mass_del = []
jet_btag_del = []
parton_pt_del = []
parton_phi_del = []
parton_eta_del = []
parton_mass_del = []

with np.load("output/ttbar_output/event_record_deltaR.npz", allow_pickle=True) as delta:
    N_match_top_in_event_del = delta['N_match_top_in_event'][:]
    jet_parton_index_del = delta['jet_parton_index'][:]
    parton_jet_index_del = delta['parton_jet_index'][:]
    jet_pt_del = delta['jet_pt'][:]
    jet_phi_del = delta['jet_phi'][:]
    jet_eta_del = delta['jet_eta'][:]
    jet_barcode_del = delta['jet_barcode'][:]
    jet_mass_del = delta['jet_mass'][:]
    jet_btag_del = delta['jet_btag'][:]
    parton_pt_del = delta['parton_pt'][:]
    parton_phi_del = delta['parton_phi'][:]
    parton_eta_del = delta['parton_eta'][:]
    parton_mass_del = delta['parton_mass'][:]
    

correct_1_chi2 = np.zeros(len(jet_pt_chi2))
correct_2_chi2 = np.zeros(len(jet_pt_chi2))
non_matched_chi2_1 = np.zeros(len(jet_pt_chi2))
non_matched_chi2_2 = np.zeros(len(jet_pt_chi2))
correct_1_chi2_mtop = np.zeros(len(jet_pt_chi2))
correct_2_chi2_mtop = np.zeros(len(jet_pt_chi2))
non_matched_chi2_1_mtop = np.zeros(len(jet_pt_chi2))
non_matched_chi2_2_mtop = np.zeros(len(jet_pt_chi2))
case = np.zeros(len(jet_pt_chi2))
case_mtop = np.zeros(len(jet_pt_chi2))
pair_info_chi2 = []
pair_info_del = []
for i in range(len(case)):
    case[i] = 99999
    case_mtop[i] = 99999

# case 1 : both correct 
# case 2 : 1 correct + 1 incorrect
# case 3 : both incorrect (but not non-matched)
# case 4 : correct/incorrect/non-matched + non-matched 

for i in tqdm.trange(len(jet_parton_index_del)):
    deltaR_vector = jet_parton_index_del[i]
    chi2_vector = jet_parton_index_chi2[i]

    b_1_del, b_2_del, q_1_del, q_2_del, q_3_del, q_4_del = 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'
    b_1_chi2, b_2_chi2, q_1_chi2, q_2_chi2, q_3_chi2, q_4_chi2 = 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'
    for j in range(len(deltaR_vector)):
        

        if deltaR_vector[j] == 0:
            b_1_del = j
        else : pass

        if deltaR_vector[j] == 1:
            q_1_del = j
        else : pass

        if deltaR_vector[j] == 2:
            q_2_del = j
        else : pass

        if deltaR_vector[j] == 3:
            b_2_del = j
        else : pass

        if deltaR_vector[j] == 4:
            q_3_del = j
        else : pass

        if deltaR_vector[j] == 5:
            q_4_del = j
        else : pass

        if chi2_vector[j] == 0:
            b_1_chi2 = j
        else : pass

        if chi2_vector[j] == 1:
            q_1_chi2 = j
        else : pass

        if chi2_vector[j] == 2:
            q_2_chi2 = j
        else : pass

        if chi2_vector[j] == 3:
            b_2_chi2 = j
        else : pass

        if chi2_vector[j] == 4:
            q_3_chi2 = j
        else : pass

        if chi2_vector[j] == 5:
            q_4_chi2 = j
        else : pass

    if np.isnan(b_1_chi2) == False or np.isnan(q_1_chi2) == False or np.isnan(q_2_chi2) == False:
        if np.isnan(b_2_chi2) == False or np.isnan(q_3_chi2) == False or np.isnan(q_4_chi2) == False:
            if b_1_chi2 == b_1_del and ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or ( q_1_chi2 == q_2_del and q_2_chi2 == q_1_del) ):
                correct_1_chi2[i] = 1
            elif b_1_chi2 == b_1_del and ( ((q_1_chi2 == q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_1_del and q_2_chi2 == q_2_del)) 
                    or ((q_1_chi2 == q_2_del and q_2_chi2 != q_1_del) or (q_1_chi2 != q_2_del and q_2_chi2 == q_1_del)) 
                    or ((q_1_chi2 != q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_2_del and q_2_chi2 != q_1_del)) ):
                non_matched_chi2_1[i] = 1
            else : pass

            if b_2_chi2 == b_2_del and ( (q_3_chi2 == q_3_del and q_4_chi2 == q_4_del) or ( q_3_chi2 == q_4_del and q_4_chi2 == q_3_del )):
                correct_2_chi2[i] = 1
            elif b_2_chi2 == b_2_del and ( ((q_3_chi2 == q_3_del and q_4_chi2 != q_4_del) or (q_3_chi2 != q_3_del and q_4_chi2 != q_4_del)) 
                    or ((q_3_chi2 == q_4_del and q_4_chi2 != q_3_del) or (q_3_chi2 != q_4_del and q_4_chi2 != q_3_del)) 
                    or ((q_3_chi2 != q_3_del and q_4_chi2 != q_4_del) or (q_3_chi2 != q_4_del and q_4_chi2 != q_3_del)) ):
                non_matched_chi2_2[i] = 1
            else: pass 

            if b_1_chi2 == b_2_del and ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del) ):
                correct_1_chi2[i] = 1
            elif b_1_chi2 == b_2_del and ( ( (q_1_chi2 == q_3_del and q_2_chi2 != q_4_del)  or (q_1_chi2 != q_3_del and q_2_chi2 == q_4_del) ) 
                    or ( (q_1_chi2 == q_4_del and q_2_chi2 != q_3_del)  or (q_1_chi2 != q_4_del and q_2_chi2 == q_3_del) ) ):
                non_matched_chi2_1[i] = 1
            else : pass 

            if b_2_chi2 == b_1_del and ( (q_3_chi2 == q_1_del and q_4_chi2 == q_2_del) or (q_3_chi2 == q_2_del and q_4_chi2 == q_1_del) ):
                correct_2_chi2[i] = 1 
            elif b_2_chi2 == b_1_del and ( ( (q_3_chi2 == q_1_del and q_4_chi2 != q_2_del)  or (q_3_chi2 != q_1_del and q_4_chi2 == q_2_del) )
                    or ( (q_3_chi2 == q_2_del and q_4_chi2 != q_1_del)  or (q_3_chi2 != q_2_del and q_4_chi2 == q_1_del) ) ):
                non_matched_chi2_2[i] = 1
            else : pass

            if (b_1_chi2 != b_1_del and b_1_chi2 != b_2_del):
                non_matched_chi2_1[i] = 1
            
            if (b_2_chi2 != b_2_del and b_2_chi2 != b_1_del):
                non_matched_chi2_2[i] = 1
            
            if (b_1_chi2 == b_1_del and b_2_chi2 == b_2_del):
                if ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or  (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del)):
                    correct_1_chi2[i] = 0
                elif ((q_3_chi2 == q_1_del and q_4_chi2 == q_2_del) or (q_3_chi2 == q_2_del and q_4_chi2 == q_1_del)):
                    correct_2_chi2[i] = 0
                else: pass
            elif (b_1_chi2 == b_2_del and b_2_chi2 == b_1_del):
                if ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or  (q_1_chi2 == q_2_del and q_2_chi2 == q_1_del)):
                    correct_1_chi2[i] = 0
                elif ((q_3_chi2 == q_3_del and q_4_chi2 == q_4_del) or (q_3_chi2 == q_4_del and q_4_chi2 == q_3_del)):
                    correct_2_chi2[i] = 0
                else: pass
        else:   
            non_matched_chi2_2[i] = 1
            correct_2_chi2[i] = 0
        
        if b_1_chi2 == b_1_del and ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or ( q_1_chi2 == q_2_del and q_2_chi2 == q_1_del) ):
                correct_1_chi2[i] = 1
        elif b_1_chi2 == b_1_del and ( ((q_1_chi2 == q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_1_del and q_2_chi2 == q_2_del)) 
                    or ((q_1_chi2 == q_2_del and q_2_chi2 != q_1_del) or (q_1_chi2 != q_2_del and q_2_chi2 == q_1_del)) 
                    or ((q_1_chi2 != q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_2_del and q_2_chi2 != q_1_del)) ):
                non_matched_chi2_1[i] = 1
        else : pass

        if b_1_chi2 == b_2_del and ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del) ):
                correct_1_chi2[i] = 1 
        elif b_1_chi2 == b_2_del and ( ( (q_1_chi2 == q_3_del and q_2_chi2 != q_4_del)  or (q_1_chi2 != q_3_del and q_2_chi2 == q_4_del) ) 
                    or ( (q_1_chi2 == q_4_del and q_2_chi2 != q_3_del)  or (q_1_chi2 != q_4_del and q_2_chi2 == q_3_del) ) ):
                non_matched_chi2_1[i] = 1
        else : pass 
        
        if (b_1_chi2 != b_1_del and b_1_chi2 != b_2_del):
                non_matched_chi2_1[i] = 1

        cand_1 = set([b_1_chi2, q_1_chi2, q_2_chi2])
        cand_2 = set([b_2_chi2, q_3_chi2, q_4_chi2])
        del_1 = set([b_1_del, q_1_del, q_2_del])
        del_2 = set([b_2_del, q_3_del, q_4_del])

        if bool(del_1.symmetric_difference(cand_1)) == False:
            if non_matched_chi2_1[i] == 1:
                non_matched_chi2_1[i] = 0
            else : 
                pass
        elif bool(del_1.symmetric_difference(cand_2)) == False:
            if non_matched_chi2_2[i] == 1:
                non_matched_chi2_2[i] = 0
            else : 
                pass
        else : pass 

        if bool(del_2.symmetric_difference(cand_2)) == False:
            if non_matched_chi2_2[i] == 1:
                non_matched_chi2_2[i] = 0
            else : 
                pass
        elif bool(del_2.symmetric_difference(cand_1)) == False:
            if non_matched_chi2_1[i] == 1:
                non_matched_chi2_1[i] = 0
            else :
                pass
        else : pass 

    else:
        non_matched_chi2_1[i] = 1
        
    
    if correct_1_chi2[i] == 1 and correct_2_chi2[i] == 1 and non_matched_chi2_1[i] == 0 and non_matched_chi2_2[i] == 0:
        case[i] = 1
    elif  ( ( correct_1_chi2[i] == 1 and correct_2_chi2[i] == 0) or ( correct_1_chi2[i] == 0 and correct_2_chi2[i] == 1) ) and (non_matched_chi2_1[i] == 0 and non_matched_chi2_2[i] == 0) :
        case[i] = 2
    elif ( ( correct_1_chi2[i] == 0 and correct_2_chi2[i] == 0) and (non_matched_chi2_1[i] == 0 and non_matched_chi2_2[i] == 0) ):
        case[i] = 3
    elif (non_matched_chi2_1[i] == 1 or non_matched_chi2_2[i] == 1):
        case[i] = 4
    else:
        print("error occur!")
    
    pair_del = [b_1_del, q_1_del, q_2_del, b_2_del, q_3_del, q_4_del]
    pair_chi2 = [b_1_chi2,  q_1_chi2, q_2_chi2, b_2_chi2, q_3_chi2, q_4_chi2]
    pair_info_del.append(pair_del)
    pair_info_chi2.append(pair_chi2)
    # print("\n")
    # print(pair_del, pair_chi2,"\n", correct_1_chi2[i], correct_2_chi2[i], non_matched_chi2_1[i], non_matched_chi2_2[i], case[i])
    
# case 1 : both correct 
# case 2 : 1 correct + 1 incorrect
# case 3 : both incorrect (but not non-matched)
# case 4 : correct/incorrect/non-matched + non-matched 

for i in tqdm.trange(len(jet_parton_index_del)):
    deltaR_vector = jet_parton_index_del[i]
    chi2_vector = jet_parton_index_chi2_mtop[i]

    b_1_del, b_2_del, q_1_del, q_2_del, q_3_del, q_4_del = 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'
    b_1_chi2, b_2_chi2, q_1_chi2, q_2_chi2, q_3_chi2, q_4_chi2 = 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'
    for j in range(len(deltaR_vector)):
        

        if deltaR_vector[j] == 0:
            b_1_del = j
        else : pass

        if deltaR_vector[j] == 1:
            q_1_del = j
        else : pass

        if deltaR_vector[j] == 2:
            q_2_del = j
        else : pass

        if deltaR_vector[j] == 3:
            b_2_del = j
        else : pass

        if deltaR_vector[j] == 4:
            q_3_del = j
        else : pass

        if deltaR_vector[j] == 5:
            q_4_del = j
        else : pass

        if chi2_vector[j] == 0:
            b_1_chi2 = j
        else : pass

        if chi2_vector[j] == 1:
            q_1_chi2 = j
        else : pass

        if chi2_vector[j] == 2:
            q_2_chi2 = j
        else : pass

        if chi2_vector[j] == 3:
            b_2_chi2 = j
        else : pass

        if chi2_vector[j] == 4:
            q_3_chi2 = j
        else : pass

        if chi2_vector[j] == 5:
            q_4_chi2 = j
        else : pass

    if np.isnan(b_1_chi2) == False or np.isnan(q_1_chi2) == False or np.isnan(q_2_chi2) == False:
        if np.isnan(b_2_chi2) == False or np.isnan(q_3_chi2) == False or np.isnan(q_4_chi2) == False:
            if b_1_chi2 == b_1_del and ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or ( q_1_chi2 == q_2_del and q_2_chi2 == q_1_del) ):
                correct_1_chi2_mtop[i] = 1
            elif b_1_chi2 == b_1_del and ( ((q_1_chi2 == q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_1_del and q_2_chi2 == q_2_del)) 
                    or ((q_1_chi2 == q_2_del and q_2_chi2 != q_1_del) or (q_1_chi2 != q_2_del and q_2_chi2 == q_1_del)) 
                    or ((q_1_chi2 != q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_2_del and q_2_chi2 != q_1_del)) ):
                non_matched_chi2_1_mtop[i] = 1
            else : pass

            if b_2_chi2 == b_2_del and ( (q_3_chi2 == q_3_del and q_4_chi2 == q_4_del) or ( q_3_chi2 == q_4_del and q_4_chi2 == q_3_del )):
                correct_2_chi2_mtop[i] = 1
            elif b_2_chi2 == b_2_del and ( ((q_3_chi2 == q_3_del and q_4_chi2 != q_4_del) or (q_3_chi2 != q_3_del and q_4_chi2 != q_4_del)) 
                    or ((q_3_chi2 == q_4_del and q_4_chi2 != q_3_del) or (q_3_chi2 != q_4_del and q_4_chi2 != q_3_del)) 
                    or ((q_3_chi2 != q_3_del and q_4_chi2 != q_4_del) or (q_3_chi2 != q_4_del and q_4_chi2 != q_3_del)) ):
                non_matched_chi2_2_mtop[i] = 1
            else: pass 

            if b_1_chi2 == b_2_del and ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del) ):
                correct_1_chi2_mtop[i] = 1
            elif b_1_chi2 == b_2_del and ( ( (q_1_chi2 == q_3_del and q_2_chi2 != q_4_del)  or (q_1_chi2 != q_3_del and q_2_chi2 == q_4_del) ) 
                    or ( (q_1_chi2 == q_4_del and q_2_chi2 != q_3_del)  or (q_1_chi2 != q_4_del and q_2_chi2 == q_3_del) ) ):
                non_matched_chi2_1_mtop[i] = 1
            else : pass 

            if b_2_chi2 == b_1_del and ( (q_3_chi2 == q_1_del and q_4_chi2 == q_2_del) or (q_3_chi2 == q_2_del and q_4_chi2 == q_1_del) ):
                correct_2_chi2_mtop[i] = 1 
            elif b_2_chi2 == b_1_del and ( ( (q_3_chi2 == q_1_del and q_4_chi2 != q_2_del)  or (q_3_chi2 != q_1_del and q_4_chi2 == q_2_del) )
                    or ( (q_3_chi2 == q_2_del and q_4_chi2 != q_1_del)  or (q_3_chi2 != q_2_del and q_4_chi2 == q_1_del) ) ):
                non_matched_chi2_2_mtop[i] = 1
            else : pass

            if (b_1_chi2 != b_1_del and b_1_chi2 != b_2_del):
                non_matched_chi2_1_mtop[i] = 1
            
            if (b_2_chi2 != b_2_del and b_2_chi2 != b_1_del):
                non_matched_chi2_2_mtop[i] = 1
            
            if (b_1_chi2 == b_1_del and b_2_chi2 == b_2_del):
                if ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or  (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del)):
                    correct_1_chi2_mtop[i] = 0
                elif ((q_3_chi2 == q_1_del and q_4_chi2 == q_2_del) or (q_3_chi2 == q_2_del and q_4_chi2 == q_1_del)):
                    correct_2_chi2_mtop[i] = 0
                else: pass
            elif (b_1_chi2 == b_2_del and b_2_chi2 == b_1_del):
                if ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or  (q_1_chi2 == q_2_del and q_2_chi2 == q_1_del)):
                    correct_1_chi2_mtop[i] = 0
                elif ((q_3_chi2 == q_3_del and q_4_chi2 == q_4_del) or (q_3_chi2 == q_4_del and q_4_chi2 == q_3_del)):
                    correct_2_chi2_mtop[i] = 0
                else: pass
        else:   
            non_matched_chi2_2_mtop[i] = 1
            correct_2_chi2_mtop[i] = 0
        
        if b_1_chi2 == b_1_del and ( (q_1_chi2 == q_1_del and q_2_chi2 == q_2_del) or ( q_1_chi2 == q_2_del and q_2_chi2 == q_1_del) ):
                correct_1_chi2_mtop[i] = 1
        elif b_1_chi2 == b_1_del and ( ((q_1_chi2 == q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_1_del and q_2_chi2 == q_2_del)) 
                    or ((q_1_chi2 == q_2_del and q_2_chi2 != q_1_del) or (q_1_chi2 != q_2_del and q_2_chi2 == q_1_del)) 
                    or ((q_1_chi2 != q_1_del and q_2_chi2 != q_2_del) or (q_1_chi2 != q_2_del and q_2_chi2 != q_1_del)) ):
                non_matched_chi2_1_mtop[i] = 1
        else : pass

        if b_1_chi2 == b_2_del and ( (q_1_chi2 == q_3_del and q_2_chi2 == q_4_del) or (q_1_chi2 == q_4_del and q_2_chi2 == q_3_del) ):
                correct_1_chi2_mtop[i] = 1 
        elif b_1_chi2 == b_2_del and ( ( (q_1_chi2 == q_3_del and q_2_chi2 != q_4_del)  or (q_1_chi2 != q_3_del and q_2_chi2 == q_4_del) ) 
                    or ( (q_1_chi2 == q_4_del and q_2_chi2 != q_3_del)  or (q_1_chi2 != q_4_del and q_2_chi2 == q_3_del) ) ):
                non_matched_chi2_1_mtop[i] = 1
        else : pass 
        
        if (b_1_chi2 != b_1_del and b_1_chi2 != b_2_del):
                non_matched_chi2_1_mtop[i] = 1

        cand_1 = set([b_1_chi2, q_1_chi2, q_2_chi2])
        cand_2 = set([b_2_chi2, q_3_chi2, q_4_chi2])
        del_1 = set([b_1_del, q_1_del, q_2_del])
        del_2 = set([b_2_del, q_3_del, q_4_del])

        if bool(del_1.symmetric_difference(cand_1)) == False:
            if non_matched_chi2_1_mtop[i] == 1:
                non_matched_chi2_1_mtop[i] = 0
            else : 
                pass
        elif bool(del_1.symmetric_difference(cand_2)) == False:
            if non_matched_chi2_2_mtop[i] == 1:
                non_matched_chi2_2_mtop[i] = 0
            else : 
                pass
        else : pass 

        if bool(del_2.symmetric_difference(cand_2)) == False:
            if non_matched_chi2_2_mtop[i] == 1:
                non_matched_chi2_2_mtop[i] = 0
            else : 
                pass
        elif bool(del_2.symmetric_difference(cand_1)) == False:
            if non_matched_chi2_1_mtop[i] == 1:
                non_matched_chi2_1_mtop[i] = 0
            else :
                pass
        else : pass 

    else:
        non_matched_chi2_1_mtop[i] = 1
        
    
    if correct_1_chi2_mtop[i] == 1 and correct_2_chi2_mtop[i] == 1 and non_matched_chi2_1_mtop[i] == 0 and non_matched_chi2_2_mtop[i] == 0:
        case_mtop[i] = 1
    elif  ( ( correct_1_chi2_mtop[i] == 1 and correct_2_chi2_mtop[i] == 0) or ( correct_1_chi2_mtop[i] == 0 and correct_2_chi2_mtop[i] == 1) ) and (non_matched_chi2_1_mtop[i] == 0 and non_matched_chi2_2_mtop[i] == 0) :
        case_mtop[i] = 2
    elif ( ( correct_1_chi2_mtop[i] == 0 and correct_2_chi2_mtop[i] == 0) and (non_matched_chi2_1_mtop[i] == 0 and non_matched_chi2_2_mtop[i] == 0) ):
        case_mtop[i] = 3
    elif (non_matched_chi2_1_mtop[i] == 1 or non_matched_chi2_2_mtop[i] == 1):
        case_mtop[i] = 4
    else:
        print("error occur!")
    
    pair_del = [b_1_del, q_1_del, q_2_del, b_2_del, q_3_del, q_4_del]
    pair_chi2 = [b_1_chi2,  q_1_chi2, q_2_chi2, b_2_chi2, q_3_chi2, q_4_chi2]
    pair_info_del.append(pair_del)
    pair_info_chi2.append(pair_chi2)
    # print("\n")
    # print(pair_del, pair_chi2,"\n", correct_1_chi2[i], correct_2_chi2[i], non_matched_chi2_1[i], non_matched_chi2_2[i], case[i])

100%|██████████| 18312/18312 [00:02<00:00, 8648.84it/s]


In [2]:
purity_event = np.sum(case == 1)/ (np.sum(case == 1) + np.sum(case == 2) + np.sum(case == 3))
purity_top = (2*np.sum(case == 1) + np.sum(case == 2))/ (2* (np.sum(case == 1) + np.sum(case == 2) + np.sum(case == 3)))
print("purity_event in origin chi2 method: {0}, purity_top: {1}".format(purity_event, purity_top))

purity_event in origin chi2 method: 0.5526709935097354, purity_top: 0.618072890664004


In [3]:
purity_event_mtop = np.sum(case_mtop == 1)/ (np.sum(case_mtop == 1) + np.sum(case_mtop == 2) + np.sum(case_mtop == 3))
purity_top_mtop = (2*np.sum(case_mtop == 1) + np.sum(case_mtop == 2))/ (2* (np.sum(case_mtop == 1) + np.sum(case_mtop == 2) + np.sum(case_mtop == 3)))
print("purity_event in mtop base chi2 method: {0}, purity_top: {1}".format(purity_event_mtop, purity_top_mtop))

purity_event in mtop base chi2 method: 0.6530189168178284, purity_top: 0.7354236848924592
